In [ ]:
import pymongo
from redis import Redis

REDIS = Redis(host='this_redis')
mongo_client = pymongo.MongoClient('this_mongo')
corpus_db = mongo_client.corpus

In [ ]:
cd ..

In [ ]:
import pandas as pd

In [ ]:
corpus = pd.read_pickle('data/corpus.p')

In [ ]:
corpus_df_records = corpus.to_dict('records')

In [ ]:
corpus_db.documents.drop()
corpus_db.documents.insert_many(corpus_df_records)

In [ ]:
corpus_db.documents.count()

In [ ]:
corpus_db.documents.find_one({'tokens' : {'$exists': False}})

# `MAPPER`

In [ ]:
def tokenize(document):
    return (document
            .replace(',','')
            .replace('.','')
            .split())

def MAPPER(document):
    for word in tokenize(document):
        yield (word, 1)

In [ ]:
doc = corpus_db.documents.find_one({'tokens' : {'$exists': False}})
list(MAPPER(doc['sentence']))

In [ ]:
unprocessed_filter = {'processed' : {'$exists': False}}
doc = corpus_db.documents.find_one(unprocessed_filter)
while doc:
    id_filter = { '_id' : doc['_id'] }
    tokens = list(MAPPER(doc['sentence']))
    update = { '$set' : {'tokens' : tokens, 'processed' : 'tokenized'} }
    corpus_db.documents.update_one(id_filter, update)
    doc = corpus_db.documents.find_one({'tokens' : {'$exists': False}})

In [ ]:
corpus_db.documents.find_one()

# `COLLECTOR`

In [ ]:
def COLLECTOR(document, vocabulary):
    for token in doc['tokens']:
        REDIS.sadd(vocabulary, token[0])
        REDIS.rpush(*token)

In [ ]:
tokenized_filter = {'processed' : 'tokenized'}
doc = corpus_db.documents.find_one(tokenized_filter)
doc

In [ ]:
while doc:
    id_filter = { '_id' : doc['_id'] }
    tokens = doc['tokens']
    update = { '$set' : {'processed' : 'counted'} }
    COLLECTOR(doc, 'corpus_vocab')
    corpus_db.documents.update_one(id_filter, update)
    doc = corpus_db.documents.find_one(tokenized_filter)

In [ ]:
vocabulary = REDIS.smembers('corpus_vocab')
list(vocabulary)[:5]

In [ ]:
for word in list(vocabulary)[:5]:
    print(REDIS.lrange(word, 0, -1))

# `REDUCER`

In [ ]:
def REDUCER(word):
    counts = [int(i) for i in REDIS.lrange(word, 0, -1)]
    return sum(counts)

In [ ]:
word_counts = []
for word in vocabulary:
    word_counts.append((word.decode(), REDUCER(word)))

In [ ]:
word_counts.sort(key=lambda x: x[1], reverse=True)

In [ ]:
word_counts[:10]